In [95]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

### Dataset X

In [96]:
SERVICE_ACCOUNT_FILE = 'key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open('Recheck HC').worksheet("20250701-0831")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [97]:
df_2025 = df.iloc[:, [2,3,17,23,24,25,26]].copy()
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   application_name     48949 non-null  object
 1   call_id              48949 non-null  object
 2   text                 48949 non-null  object
 3   manual_audit_result  48949 non-null  object
 4   checkbox             48949 non-null  object
 5   checker              48949 non-null  object
 6   checking_date        48949 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB


### Dataset Y

In [98]:
df_recheck = pd.read_csv('dataset_qc/recheck_hc.csv')

df_recheck = df_recheck.iloc[:, [2,3,4,8,19,27]]
df_recheck

cols = ['Nama Agent ', 'ASI/AFI', 'Efektif ']

# Date
df_recheck['Tanggal Pengerjaan'] = pd.to_datetime(df_recheck['Tanggal Pengerjaan'])

# Cols Cat
for col in cols:
    df_recheck[col] = df_recheck[col].astype('category')

# count feature clean
df_recheck['Count Efektif'] = df_recheck['Count Efektif'].fillna(0).astype(int)

df_recheck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31158 entries, 0 to 31157
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Tanggal Pengerjaan  31158 non-null  datetime64[ns]
 1   Nama Agent          31158 non-null  category      
 2   ASI/AFI             31158 non-null  category      
 3   Call ID             31158 non-null  object        
 4   Efektif             31158 non-null  category      
 5   Count Efektif       31158 non-null  int64         
dtypes: category(3), datetime64[ns](1), int64(1), object(1)
memory usage: 822.2+ KB


C:\Users\ozy.prazuganda\AppData\Local\Temp\ipykernel_14076\2325867966.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_recheck = pd.read_csv('dataset_qc/recheck_hc.csv')


### X + Y

In [99]:
hc = df_recheck[
    (df_recheck['Tanggal Pengerjaan'] < '2025-12-03') &
    (df_recheck['Count Efektif'] != 1)
]

hc.drop_duplicates('Call ID ', keep='first').reset_index(drop=True)

,Tanggal Pengerjaan,Nama Agent,ASI/AFI,Call ID,Efektif,Count Efektif
0,2025-11-11,Azer,ASI,7f20d864c6224ed3bff2c2c3fc4f79e4,Miss Target/ Not HC,0
1,2025-11-11,Azer,ASI,4b3cc66733b3457aa8a04e6b405dac93,Miss Target/ Not HC,0
2,2025-11-11,Azer,ASI,9d65e5a2b2f04b37803730f62b1085eb,Miss Target/ Not HC,0
3,2025-11-11,Azer,ASI,fb3b6ad4ffa041eb9d13cb44b2c54bb0,Miss Target/ Not HC,0
4,2025-11-11,Azer,ASI,fc0859eabc0641eb9046d4ccb36a7139,Miss Target/ Not HC,0
...,...,...,...,...,...,...
406,2025-12-02,Neneng,AFI,195cbe6e7a70473ca50dbfcaf43f991b,Miss Target/ Not HC,0
407,2025-12-02,Neneng,ASI,24e4e65a669e49ca8b25030e7f23201a,Miss Target/ Not HC,0
408,2025-12-02,Neneng,ASI,5088c4ae79e04533a234793d2d76c836,Miss Target/ Not HC,0
409,2025-12-02,Neneng,AFI,47099efa5a8341cd9091f929253b623d,Miss Target/ Not HC,0
